# Notebook 1: Create lineage-specific Genome Scale Metabolic Models

In [1]:
#import package needed
import cobra
import pandas as pd
from cobra.io import load_json_model
import os
from os.path import join
from glob import glob
from cobra.manipulation.delete import delete_model_genes, remove_genes
import numpy as np
import copy as cp

In [2]:
# Directory stuff
mod_dir = '../data/models/'
res_dir = '../results/'
del_dir = '../data/deletions/'
snp_dir = '../data/snps/'
built_mod_dir = res_dir + 'built_models/'
delsSGsnps_mod_dir = built_mod_dir + 'delsSGsnps_mods/'
delsAllsnps_mod_dir = built_mod_dir + 'delsAllsnps_mods/'

In [3]:
# Create output directories if they don't exist
if not os.path.isdir(res_dir):
    os.mkdir(res_dir)
    print('%s created'%res_dir)
if not os.path.isdir(built_mod_dir):
    os.mkdir(built_mod_dir)
    print('%s created'%built_mod_dir)
if not os.path.isdir(delsSGsnps_mod_dir):
    os.mkdir(delsSGsnps_mod_dir)
    print('%s created'%delsSGsnps_mod_dir)
if not os.path.isdir(delsAllsnps_mod_dir):
    os.mkdir(delsAllsnps_mod_dir)
    print('%s created'%delsAllsnps_mod_dir)

### Load base reconstruction (iEK1011 2.0)

In [4]:
base_mod = cobra.io.load_json_model(join(mod_dir, "iEK1011_2.0.json"))

### Get a matrix of genes to remove per lineage based on the statistical analysis done with the 35000 genomes analized (deleted genes and genes with stop gain SNPs)

In [5]:
# Import deletion files 
delFiles = os.listdir(del_dir)
delFiles = [file for file in delFiles if '.txt' in file]

In [6]:
# Get a list of lineages 
lins = [x[0:2] for x in delFiles]

In [7]:
mod_genes = [x.id for x in base_mod.genes]
pd.Series(mod_genes).to_csv(res_dir + 'iEK1011_2.0_genes.txt', sep = '\t', index = False, header = False)

In [8]:
# Print genes in iEK1011_2.0 to del to generate each lineage. 
for i in range(0, len(delFiles)):
    file = del_dir + delFiles[i]
    genes2Del = pd.read_csv(file, header = None).iloc[:, 0].tolist()
    print(delFiles[i][0:2], genes2Del)

A1 ['Rv0221', 'Rv0222', 'Rv0223c', 'Rv2074', 'Rv2349c', 'Rv2350c', 'Rv2351c', 'Rv3617', 'Rv3737']
A2 ['Rv0221', 'Rv0222', 'Rv0223c', 'Rv2074', 'Rv3617']
A3 ['Rv0032', 'Rv0221', 'Rv0222', 'Rv0223c', 'Rv1755c', 'Rv1760', 'Rv2074', 'Rv2350c', 'Rv2351c', 'Rv3113', 'Rv3116', 'Rv3117', 'Rv3119', 'Rv3617']
A4 ['Rv0221', 'Rv0222', 'Rv0223c', 'Rv1257c', 'Rv1511', 'Rv1512', 'Rv2074', 'Rv2349c', 'Rv2350c', 'Rv2351c', 'Rv3117', 'Rv3119', 'Rv3617']
L1 ['Rv1525']
L2 ['Rv0072', 'Rv0073', 'Rv1755c', 'Rv1760']
L3 ['Rv0815c', 'Rv3117', 'Rv3516']
L5 ['Rv2074', 'Rv3513c']
L6 ['Rv0221', 'Rv0222', 'Rv0223c', 'Rv0267', 'Rv2074', 'Rv3617']
L7 ['Rv2291', 'Rv3468c', 'Rv3469c', 'Rv3513c']
L8 ['Rv0075', 'Rv0113', 'Rv0114', 'Rv2349c', 'Rv2948c', 'Rv3281']
L9 ['Rv0221', 'Rv0222', 'Rv0223c', 'Rv0267', 'Rv2074', 'Rv3617']


In [9]:
# Create the matrix 
delMat = pd.DataFrame(index = mod_genes, columns = lins)
for i in range(0, len(delFiles)):
    file = del_dir + delFiles[i]
    genes2Del = pd.read_csv(file, header = None).iloc[:, 0].tolist()
    delList = []
    for idx in list(delMat.index):
        if idx in genes2Del:
            delList.append(0.0)
        else:
            delList.append(1.0)
    delMat[lins[i]] = delList
delMat.insert(7, 'L4', [1.0]*len(mod_genes))
delMat.to_csv(res_dir + 'delMat.csv')

In [10]:
# Add stop gain SNPs to the matrix.
for i in range(0, delMat.shape[1]):
    lin = delMat.columns[i]
    filePath = snp_dir + '%s_stopGain.txt'%lin
    if os.path.exists(filePath) and os.stat(filePath).st_size != 0:
        snps = pd.read_csv(filePath, sep = '\t', header = None)[0].tolist()
        for j in range(0, delMat.shape[0]):
            gene = delMat.index[j]
            if gene in snps and delMat[lin][gene] != 0.0:
                print('Changing %s from intact to deleted in lineage %s.'%(gene, lin))
                delMat[lin][gene] = 0.0
            elif gene in snps and delMat[lin][gene] == 0.0: 
                print('%s is already deleted in lineage %s.'%(gene, lin))
    else:
        print("%s doesn't have pontentially deletereous SNPs."%lin)
            
delMat.to_csv(res_dir + 'del_sgSNPs_mat.csv')

A1 doesn't have pontentially deletereous SNPs.
A2 doesn't have pontentially deletereous SNPs.
Changing Rv3468c from intact to deleted in lineage A3.
Changing Rv2187 from intact to deleted in lineage A3.
Changing Rv0132c from intact to deleted in lineage A3.
Changing Rv2958c from intact to deleted in lineage A4.
Changing Rv1745c from intact to deleted in lineage A4.
L1 doesn't have pontentially deletereous SNPs.
L2 doesn't have pontentially deletereous SNPs.
L3 doesn't have pontentially deletereous SNPs.
L4 doesn't have pontentially deletereous SNPs.
Changing Rv2850c from intact to deleted in lineage L5.
L6 doesn't have pontentially deletereous SNPs.
Changing Rv0889c from intact to deleted in lineage L7.
Changing Rv3084 from intact to deleted in lineage L7.
Changing Rv2860c from intact to deleted in lineage L8.
Changing Rv2713 from intact to deleted in lineage L8.
Changing Rv2505c from intact to deleted in lineage L8.
Changing Rv3378c from intact to deleted in lineage L8.
Changing Rv182

### Get a different matrix that include deletions, stop gained SNPs and missense SNPs predicted to be deletereous with PROVEAN. 

In [11]:
# Copy delMat (deletions + stop gain SNPs) and add missense provean significative SNPs
delAllSNPsMat = cp.copy(delMat)
for i in range(0, delAllSNPsMat.shape[1]):
    lin = delAllSNPsMat.columns[i]
    filePath = snp_dir + '%s_missProv.txt'%lin
    if os.path.exists(filePath) and os.stat(filePath).st_size != 0:
        snps = pd.read_csv(filePath, sep = '\t', header = None)[0].tolist()
        for j in range(0, delAllSNPsMat.shape[0]):
            gene = delAllSNPsMat.index[j]
            if gene in snps and delAllSNPsMat[lin][gene] != 0.0:
                print('Changing %s from intact to deleted in lineage %s.'%(gene, lin))
                delAllSNPsMat[lin][gene] = 0.0
            elif gene in snps and delAllSNPsMat[lin][gene] == 0.0: 
                print('%s is already deleted in lineage %s.'%(gene, lin))
    else:
        print("%s doesn't have pontentially deletereous SNPs."%lin)
            
delAllSNPsMat.to_csv(res_dir + 'del_AllSNPs_mat.csv')

Changing Rv2222c from intact to deleted in lineage A1.
Changing Rv2471 from intact to deleted in lineage A1.
Changing Rv1933c from intact to deleted in lineage A1.
Changing Rv0896 from intact to deleted in lineage A1.
Changing Rv0512 from intact to deleted in lineage A1.
Changing Rv0206c from intact to deleted in lineage A1.
Changing Rv1451 from intact to deleted in lineage A1.
Changing Rv2194 from intact to deleted in lineage A1.
Changing Rv0091 from intact to deleted in lineage A1.
Changing Rv3801c from intact to deleted in lineage A1.
Changing Rv1023 from intact to deleted in lineage A1.
Changing Rv0070c from intact to deleted in lineage A1.
Changing Rv1605 from intact to deleted in lineage A1.
Changing Rv0644c from intact to deleted in lineage A1.
Changing Rv0127 from intact to deleted in lineage A1.
Changing Rv3382c from intact to deleted in lineage A1.
Changing Rv0803 from intact to deleted in lineage A1.
Changing Rv3314c from intact to deleted in lineage A1.
Changing Rv3913 from

Changing Rv1826 from intact to deleted in lineage L3.
Changing Rv2483c from intact to deleted in lineage L3.
Changing Rv2378c from intact to deleted in lineage L3.
Changing Rv2267c from intact to deleted in lineage L3.
Changing Rv0129c from intact to deleted in lineage L3.
Changing Rv0223c from intact to deleted in lineage L3.
Changing Rv0777 from intact to deleted in lineage L3.
Changing Rv0803 from intact to deleted in lineage L3.
Changing Rv1286 from intact to deleted in lineage L3.
Changing Rv1029 from intact to deleted in lineage L3.
Changing Rv0294 from intact to deleted in lineage L3.
L4 doesn't have pontentially deletereous SNPs.
Changing Rv2531c from intact to deleted in lineage L5.
Changing Rv2222c from intact to deleted in lineage L5.
Changing Rv3808c from intact to deleted in lineage L5.
Changing Rv3784 from intact to deleted in lineage L5.
Changing Rv3794 from intact to deleted in lineage L5.
Changing Rv1656 from intact to deleted in lineage L5.
Changing Rv1188 from intact

Changing Rv0206c from intact to deleted in lineage L9.
Changing Rv1451 from intact to deleted in lineage L9.
Changing Rv2194 from intact to deleted in lineage L9.
Changing Rv2124c from intact to deleted in lineage L9.
Changing Rv1550 from intact to deleted in lineage L9.
Changing Rv1484 from intact to deleted in lineage L9.
Changing Rv2524c from intact to deleted in lineage L9.
Changing Rv0070c from intact to deleted in lineage L9.
Changing Rv1605 from intact to deleted in lineage L9.
Changing Rv2933 from intact to deleted in lineage L9.
Changing Rv0644c from intact to deleted in lineage L9.
Changing Rv0904c from intact to deleted in lineage L9.
Changing Rv3199c from intact to deleted in lineage L9.
Changing Rv0137c from intact to deleted in lineage L9.
Changing Rv3148 from intact to deleted in lineage L9.
Changing Rv0392c from intact to deleted in lineage L9.
Changing Rv1155 from intact to deleted in lineage L9.
Changing Rv3410c from intact to deleted in lineage L9.
Changing Rv3314c f

### Delete genes with deletions and stop gain mutations from iEK1011 2.0 model and save them
Remove the missing genes/genes with stop gained SNPs in each lineage from iEK1011 model. 

In [12]:
#create lineage-specific draft models and save them
for lin in delMat.columns:
    
    #Get the list of Gene IDs from the homology matrix dataframe for the current strain without a homolog
    currentStrain=delMat[lin]
    nonHomologous=currentStrain[currentStrain==0.0]
    nonHomologous=nonHomologous.index.tolist()
    
    #Define a list of Gene objects from the base reconstruction to be deleted from the current strain
    toDelete=[]
    for gene in nonHomologous:
        toDelete.append(base_mod.genes.get_by_id(gene))

    #Establish a model copy and use the COBRApy function to remove the appropriate content and save this model
    modelCopy=base_mod.copy()
    remove_genes(modelCopy, toDelete, remove_reactions=True)
    modelCopy.id=str(lin)
    cobra.io.json.save_json_model(modelCopy, str(delsSGsnps_mod_dir+lin+'_delsSGsnps.json'), pretty=False)
    print('%s_delsSGsnps.json saved at %s.'%(lin, delsSGsnps_mod_dir))

A1_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.
A2_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.
A3_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.
A4_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.
L1_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.
L2_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.
L3_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.
L4_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.
L5_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.
L6_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.
L7_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.
L8_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.
L9_delsSGsnps.json saved at ../results/built_models/delsSGsnps_mods/.


In [13]:
# gather the general information on the draft models
for lin in delMat.columns:
    model=cobra.io.load_json_model(str(delsSGsnps_mod_dir+lin+'_delsSGsnps.json'))
    print (model.id,'Number of Model Genes:',len(model.genes),'Number of Model Reactions:',len(model.reactions))

A1 Number of Model Genes: 1001 Number of Model Reactions: 1236
A2 Number of Model Genes: 1005 Number of Model Reactions: 1237
A3 Number of Model Genes: 993 Number of Model Reactions: 1231
A4 Number of Model Genes: 995 Number of Model Reactions: 1229
L1 Number of Model Genes: 1009 Number of Model Reactions: 1237
L2 Number of Model Genes: 1006 Number of Model Reactions: 1236
L3 Number of Model Genes: 1007 Number of Model Reactions: 1238
L4 Number of Model Genes: 1010 Number of Model Reactions: 1238
L5 Number of Model Genes: 1007 Number of Model Reactions: 1238
L6 Number of Model Genes: 1004 Number of Model Reactions: 1237
L7 Number of Model Genes: 1004 Number of Model Reactions: 1238
L8 Number of Model Genes: 999 Number of Model Reactions: 1228
L9 Number of Model Genes: 1002 Number of Model Reactions: 1237


### Delete genes with deletions and potentially deletereous SNPs from iEK1011 2.0 model and save them
Remove the missing genes/genes with stop gain SNPs and missense SNPs predicted to be deletereous with PROVEAN in each lineage from iEK1011 model. 

In [14]:
#create lineage-specific draft models and save them
for lin in delAllSNPsMat.columns:
    
    #Get the list of Gene IDs from the homology matrix dataframe for the current strain without a homolog
    currentStrain=delAllSNPsMat[lin]
    nonHomologous=currentStrain[currentStrain==0.0]
    nonHomologous=nonHomologous.index.tolist()
    
    #Define a list of Gene objects from the base reconstruction to be deleted from the current strain
    toDelete=[]
    for gene in nonHomologous:
        toDelete.append(base_mod.genes.get_by_id(gene))

    #Establish a model copy and use the COBRApy function to remove the appropriate content and save this model
    modelCopy=base_mod.copy()
    remove_genes(modelCopy, toDelete, remove_reactions=True)
    modelCopy.id=str(lin)
    cobra.io.json.save_json_model(modelCopy, str(delsAllsnps_mod_dir+lin+'_delsAllsnps.json'), pretty=False)
    print('%s_delsAllsnps.json saved at %s.'%(lin, delsAllsnps_mod_dir))

A1_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.
A2_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.
A3_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.
A4_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.
L1_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.
L2_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.
L3_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.
L4_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.
L5_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.
L6_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.
L7_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.
L8_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.
L9_delsAllsnps.json saved at ../results/built_models/delsAllsnps_mods/.


In [15]:
# gather the general information on the draft models
for lin in delAllSNPsMat.columns:
    model=cobra.io.load_json_model(str(delsAllsnps_mod_dir+lin+'_delsAllsnps.json'))
    print (model.id,'Number of Model Genes:',len(model.genes),'Number of Model Reactions:',len(model.reactions))

A1 Number of Model Genes: 978 Number of Model Reactions: 1202
A2 Number of Model Genes: 984 Number of Model Reactions: 1221
A3 Number of Model Genes: 906 Number of Model Reactions: 1157
A4 Number of Model Genes: 959 Number of Model Reactions: 1194
L1 Number of Model Genes: 997 Number of Model Reactions: 1234
L2 Number of Model Genes: 987 Number of Model Reactions: 1209
L3 Number of Model Genes: 987 Number of Model Reactions: 1211
L4 Number of Model Genes: 1010 Number of Model Reactions: 1238
L5 Number of Model Genes: 951 Number of Model Reactions: 1186
L6 Number of Model Genes: 966 Number of Model Reactions: 1173
L7 Number of Model Genes: 944 Number of Model Reactions: 1163
L8 Number of Model Genes: 952 Number of Model Reactions: 1197
L9 Number of Model Genes: 959 Number of Model Reactions: 1186
